In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
from glob import glob

In [ ]:
Image_train = '/content/drive/MyDrive/Data/Mitosis/Private/train'
images_Train = glob(f"{Image_train}/*")
Image_test = '/content/drive/MyDrive/Data/Mitosis/Private/test/kmit'
images_Test = glob(f"{Image_test}/*")

In [ ]:
print(len(images_Train))
print(len(images_Test))

In [ ]:
# !pip install pyyaml
# !pip install pycocotools
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !git clone https://github.com/facebookresearch/detectron2.git
# %cd detectron2/
# !pip install -e .
# !pip3 install histomicstk --find-links https://girder.github.io/large_image_wheels

In [ ]:
import torch
print(torch.__version__)  # Should match the version required by the Detectron2 build
print(torch.cuda.is_available())  # Should return True if CUDA is enabled
print(torch.version.cuda)  # Should return '10.1'
import detectron2
print("Detectron2 version:", detectron2.__version__)

In [ ]:
import matplotlib.pyplot as plt
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.data.datasets import register_coco_instances

In [ ]:
Image_train = '/content/drive/MyDrive/Data/Mitosis/Private/train'
images_Train = glob(f"{Image_train}/*")
Image_test = '/content/drive/MyDrive/Data/Mitosis/Private/test/kmit'
images_Test = glob(f"{Image_test}/*")
Image_json_train = '/content/drive/MyDrive/Data/Mitosis/Private/train'
Train_Json= glob(f"{Image_json_train}/*.json")
Image_json_test = '/content/drive/MyDrive/Data/Mitosis/Private/test/kmit'
Test_Json= glob(f"{Image_json_test}/*.json")

In [ ]:
print(len(images_Train))
print(len(images_Test))
print(f"Train json File - {Train_Json}")
print(f"Test json File - {Test_Json}")

In [ ]:
register_coco_instances("train", {}, "/content/drive/MyDrive/Data/Mitosis/Private/train/train_coco.json", "/content/drive/MyDrive/Data/Mitosis/Private/train")
register_coco_instances("test", {}, "/content/drive/MyDrive/Data/Mitosis/Private/test/kmit/test_coco.json", "/content/drive/MyDrive/Data/Mitosis/Private/test/kmit")

print(MetadataCatalog.get("train"))

Metadata(name='train', json_file='/content/drive/MyDrive/Data/Mitosis/Private/train/train_coco.json', image_root='/content/drive/MyDrive/Data/Mitosis/Private/train', evaluator_type='coco')


In [ ]:
print(MetadataCatalog.get("test"))

Metadata(name='test', json_file='/content/drive/MyDrive/Data/Mitosis/Private/test/kmit/test_coco.json', image_root='/content/drive/MyDrive/Data/Mitosis/Private/test/kmit', evaluator_type='coco')


In [ ]:
dataset_dicts = DatasetCatalog.get("train")
my_dataset_train_metadata = MetadataCatalog.get("train")
import random
from detectron2.utils.visualizer import Visualizer
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    print(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

cfg = get_cfg()

In [ ]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Initialize from model zoo
cfg.OUTPUT_DIR = "/content/drive/MyDrive/Data/Mitosis/FasterRCNNResnet50FPN"

In [ ]:
cfg.INPUT.MIN_SIZE_TRAIN = 800  # Increase to enhance detection at larger scales
cfg.INPUT.MAX_SIZE_TRAIN = 1333
cfg.INPUT.MIN_SIZE_TEST = 800
cfg.INPUT.MAX_SIZE_TEST = 1333

In [ ]:
cfg1.INPUT.MIN_SIZE_TEST = 0
cfg1.INPUT.MAX_SIZE_TEST = 1024  # Increased for better handling of large objects
cfg1.INPUT.MIN_SIZE_TRAIN = 0
cfg1.INPUT.MAX_SIZE_TRAIN = 1024

In [ ]:
cfg1.DATASETS.TRAIN = ("train",)
cfg1.DATASETS.TEST = ("test",)

In [ ]:
cfg1.DATALOADER.ASPECT_RATIO_GROUPING = True


In [ ]:
cfg1.MODEL.ANCHOR_GENERATOR.SIZES = [[8, 16, 32, 64, 128]]
cfg1.MODEL.ANCHOR_GENERATOR.ANGLES = [[-90, 0, 90]]

In [ ]:
cfg1.MODEL.RPN.IOU_THRESHOLDS = [0.3, 0.7]
cfg1.MODEL.RPN.NMS_THRESH = 0.7

In [ ]:
cfg1.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.5]
cfg1.MODEL.ROI_HEADS.POSITIVE_FRACTION = 0.25
cfg1.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg1.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.7
cfg1.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512

In [ ]:
# Solver Configurations
cfg1.SOLVER.LR_SCHEDULER_NAME = "WarmupMultiStepLR"
cfg1.SOLVER.WARMUP_ITERS = 1000
cfg1.SOLVER.MAX_ITER = 5000
cfg1.SOLVER.BASE_LR = 0.0005
cfg1.SOLVER.MOMENTUM = 0.9
cfg1.SOLVER.WEIGHT_DECAY = 0.0001
cfg1.SOLVER.GAMMA = 0.1
cfg1.SOLVER.STEPS = (3000,) 
cfg1.SOLVER.CHECKPOINT_PERIOD = 500
cfg1.TEST.EVAL_PERIOD = 500
cfg1.SOLVER.IMS_PER_BATCH = 12

In [ ]:
trainer = CocoTrainer(cfg1)
trainer.resume_or_load(resume=False)

[12/10 17:18:55 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


In [ ]:
trainer.train()

In [ ]:
def visualize_predictions(predictor, image_paths, metadata):
    for image_path in image_paths:
        img = cv2.imread(image_path)
        outputs = predictor(img)
        v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.2)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        cv2.imshow("Prediction", v.get_image()[:, :, ::-1])
        cv2.waitKey(0)

In [ ]:
from detectron2.engine import DefaultPredictor
predictor_cfg = cfg1.clone()
predictor_cfg.MODEL.WEIGHTS = os.path.join(cfg1.OUTPUT_DIR, "model_final.pth")
predictor_cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(predictor_cfg)